In [ ]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

In [ ]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

In [ ]:
# Set the tickers for both the bond and stock portion of the portfolio
#SPY = S&P500
#AGG= Core US Aggregate Bond ETF
#QQQ= QQQ tracks a modified-market-cap-weighted index of 100 NASDAQ-listed stocks.
#VTV= Value ETF (Provides a convenient way to match the performance of many of the nation’s largest value stocks), 
#IEMG = Emerging Market ETF
#IWM = Russell 2000 index ETF (small cap)
tickers = ['SPY', 'AGG', 'QQQ','VTV', 'IEMG', 'IWM']

# Set timeframe to 1D 
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2017-01-01T00:00:00", tz = "America/Los_Angeles").isoformat()
end_date = pd.Timestamp("2021-12-31T00:00:00", tz = "America/Los_Angeles").isoformat()
limit_rows = 10000



In [ ]:
#function that accepts a variable string ticker for pulling the dataframe from alpaca
def puller(ticker):
    df1 = alpaca.get_bars(ticker, 
                          timeframe, 
                          start=start_date, 
                          end=end_date, 
                          limit=limit_rows).df
    return df1

#function for structuring dataframe so MCsimulation will accept as input
def structuredata(df, ticker):    
    col_names = [(ticker, x) for x in df.columns]
    df.columns = pd.MultiIndex.from_tuples(col_names)
    return df

#function to merge two dfs into MCsimulation format
def merger(df1, df2):
        df_merged = pd.merge(df1, df2, how = "inner", left_index=True, right_index=True)
        return df_merged

#run function that accepts a list of strings and other defined other variables needed for structuring the combined data
def run(list, currentdf, ticker):
        if(list.index(ticker) == 0):
                df = puller(ticker)
                df = structuredata(df, ticker)
                #display(df.head())
                #print('hi')
                return df
        else:        
                df = puller(ticker)
                df = structuredata(df, ticker)
                df = merger(currentdf, df)
                #display(df.head())
                #print('ho')
                return df

MCdf = pd.DataFrame()

for i in tickers:
        MCdf = run(tickers, MCdf, i)

display(MCdf.head())


In [ ]:
# Configure the Monte Carlo simulation to forecast 30 years cumulative returns
# The weights can be split in any fashion between tickers ['SPY', 'AGG', 'QQQ','VTV', 'IEMG', 'IWM']
# Run 500 samples.
MC_equal_weight = MCSimulation(
    portfolio_data = MCdf,
    weights = [.4, .2, .1, .1, .1, .1],
    num_simulation = 500,
    num_trading_days = 252*10
)

# Review the simulation input data
# Printing the first five rows of the simulation input data
MC_equal_weight.portfolio_data.head()

In [ ]:
MC_equal_weight.calc_cumulative_return()

In [ ]:
#MC_equal_weight_plot = MC_equal_weight.plot_simulation()
MC_equal_weight_dist = MC_equal_weight.plot_distribution()
MC_summary_stats = MC_equal_weight.summarize_cumulative_return()
print(MC_summary_stats)

In [ ]:
# Using the lower and upper `95%` confidence intervals from the summary statistics,
# calculate the range of the probable cumulative returns for a $500000 investment
ci_95_lower_cumulative_return = MC_summary_stats[8] * 500000
ci_95_upper_cumulative_return = MC_summary_stats[9] * 500000

print(f"There is a 95% chance that an initial investment of $500,000 in the portfolio"
    f" over the next 5 years will end within in the range of"
    f" ${ci_95_lower_cumulative_return: .2f} and ${ci_95_upper_cumulative_return: .2f}.")

In [ ]:
"""
Although this result presents potentially fantastic news, it’s important to note that these five-year forecasted return values are based on only three years of historical price data. The five-year forecast simulates more variability than the data that the simulation is based on includes. In general, it’s ideal to supply one year of historical data for each year of simulated data.

If we simulate using only small amounts of data during a recent time when markets are booming, or instead falling precipitously, a Monte-Carlo Analysis will inadvertently extrapolate this temporary market movement too far into the future. Getting data over a longer time period mitigates this effect. Due to the limitations of the Alpaca API, however, we can typically produce just three full years of historical data.
"""